# Burrows Delta: Eine Methode des Machine Learning zur Detektion von Autorenschaft

Ein bewährter stilometrischer Ansatz zur Erkennung von Autorenschaft ist die als Burrows Delta bezeichnete Methode, die mittels Machine Learning in einem überwachtem Verfahren mit gelabelten Trainingsdaten Texte von Autor:innen klassifiziert. Der dahinterstehende Algorithmus, der zu Beginn der 2000er-Jahre von John Burrows in die stilometrische Analyse eingeführt wurde, [^fn1] ist vergleichsweise einfach, liefert aber solide und fundierte Ergebnisse, insbesondere wenn längere Texte untersucht werden. Auch dieser Ansatz basiert auf der Berechnung von Distanzen zwischen den zu vergleichenden Texten. Im ersten Schritt werden mit den Autor:innennamen gelabelte Texte trainiert, um im zweiten Schritte neuen, unbekannten Texten das Autor:in-Label des Textes zuzuordnen, der stilistisch dem fraglichen Text am nächsten ist. Es wird also nach dem Prinzip des nächsten Nachbars (Nearest Neighbor) klassifiziert. Diese Form des Machine Learning wird auch als Instance-Based Learning oder Memory-Based Learning bezeichnet. 

Eine genaue und eingängige Beschreibung der mathematischen Herleitung von Burrows Delta bieten Karsdorp, Kestemont und Riddell in einem [Unterkapitel](https://www.humanitiesdataanalysis.org/stylometry/notebook.html#burrowss-delta) ihres einführenden Kapitels zur Stilometrie in den Digital Humanities: [Stylometry and the Voice of Hildegard](https://www.humanitiesdataanalysis.org/stylometry/notebook.html#). [^fn2]

Aus diesem [Kapitel](https://www.humanitiesdataanalysis.org/stylometry/notebook.html#computing-document-distances-with-delta) sind der Code sowie die [Daten](https://doi.org/10.5281/zenodo.3560761) entnommen, [^fn3] die nachfolgend in leicht angepasster Fassung genutzt werden, um den Einsatz von Burrows Delta zur Zuschreibung von Autorschaft zu veranschaulichen. Als historisches Fallbeispiel dienen mittelalterliche Quellen, die entweder von Hildegard von Bingen oder von einem ihrer Sekretäre verfasst wurden. Ein informatives Video zu diesem Fall steht [hier](https://vimeo.com/70881172) zur Verfügung. Das Video wurde von den Forschenden erstellt, die diese unklare Lage bei der Autorenschaft der lateinischen Quellen aus dem 12. Jahrhundert wissenschaftlich untersucht haben. [^fn4]

[^fn1]: John Burrows, 'Delta': a Measure of Stylistic Difference and a Guide to Likely Authorship, in: Literary and Linguistic Computing 17, Nr. 3 (2002), S. 267–287, https://doi.org/10.1093/llc/17.3.267.

[^fn2]: Folgert Karsdorp, Mike Kestemont, Allen Roddell, Humanities Data Analysis. Case Studies with Python, Princeton 2021, S. 248-280, hier 252-254, https://www.humanitiesdataanalysis.org/index.html

[^fn3]: Folgert Karsdorp, Mike Kestemont, Allen Roddell, Supplemental Materials for "Humanities Data Analysis" [Data set]. Zenodo, https://doi.org/10.5281/zenodo.3560761.

[^fn4]: Mike Kestemont, Sara Moens, Jeroen Deploige, Collaborative authorship in the twelfth century: A stylometric study of Hildegard of Bingen and Guibert of Gembloux, in: Digital Scholarship in the Humanities 30, Nr. 2 (2015), S. 199–224, https://doi.org/10.1093/llc/fqt063

## Importe

In [1]:
import os

import numpy as np

import sklearn.feature_extraction.text as text
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics

import scipy.spatial.distance as scidist

## Helferfunktion

### Zum Laden der Daten

Im Ordner 'texts' befinden sich drei Textkorpora: Zum einen von Hildegard von Bingen, von ihrem Sekretär Guibert von Gembloux sowie als unabhängige Vergleichsgröße von einem weiteren Zeitgenossen, Bernard von Clairvaux. Die lateinischen Textkorpora liegen bereits in lemmatisierter Form vor. Da zur stilometrischen Analyse die zu vergleichenden Texte eine einheitliche Länge aufweisen sollten, bringt die nachfolgende Funktion die Texte auf den entsprechenden Umfang. Die Länge wird bestimmt vom kürzesten der zu untersuchenden Texte. Die im Unterordner 'test' abgelegten Dateien enthalten die zu prüfenden Quellen mit den unklaren Autor:innenzuschreibungen.

In [2]:
def load_directory(directory, max_length):
    documents, authors, titles = [], [], []
    for filename in os.scandir(directory):
        if not filename.name.endswith('.txt'):
            continue
        author, _ = os.path.splitext(filename.name)

        with open(filename.path) as f:
            contents = f.read()
        lemmas = contents.lower().split()
        start_idx, end_idx, segm_cnt = 0, max_length, 1

        # extract slices from the text:
        while end_idx < len(lemmas):
            documents.append(' '.join(lemmas[start_idx:end_idx]))
            authors.append(author[0])
            title = filename.name.replace('.txt', '').split('_')[1]
            titles.append(f"{title}-{segm_cnt}")

            start_idx += max_length
            end_idx += max_length
            segm_cnt += 1

    return documents, authors, titles

## Testen und Evaluieren von Burrows Delta auf dem bekannten Korpus

### Laden der Daten

Die Textkorpora werden in Abschnitte (chunks) mit einer Länge von 10.000 Tokens gebracht. In den zurückgegebenen Listen befinden sich die Quellentexte als string (documents), der erste Buchstabe des:r Autor:in (authors) als Label für die spätere Klassifizierung sowie der restliche Teil des Dateinamens (title).

In [3]:
documents, authors, titles = load_directory('data/texts', 10000)

In [22]:
print(f'Anzahl der Dokumente: {len(documents)}')
print(authors[:5])
print(titles[:5])

Anzahl der Dokumente: 36
['B', 'B', 'B', 'B', 'B']
['ep-1', 'ep-2', 'ep-3', 'ep-4', 'ep-5']


### Erstellen des Vokabulars und der Wordvektoren

Computer verarbeiten Zahlen - daher müssen Texte in numerische Werte umgewandelt werden: Für jedes jedes Dokument in einem Textkorpus kann ein Vektor erstellt werden, der das Dokument repräsentiert. Bei stilometrischen Analysen wurde herausgefunden, dass Funktionswörter, insbesondere Stoppwörter, als starkes Signal zur Bestimmung von Autorenschaft herangezogen werden können. Funktionswörter werden von Autor:innen unbewusst in bestimmten Mustern genutzt, dadurch stellen sie eine sehr gute Signatur dar, um Autor:innen zu identifizieren. Für den vorliegenden Fall wurde eine Liste erstellt, die 65 Funktionswörter umfasst. Diese Liste wird zunächst aus der Datei wordlist.txt in die Variable vocab eingelesen. Dies ist das Vokabular, welches bei der weiteren stilometrischen Untersuchung zur Authorship Attribution genutzt wird. Mehr zum Hintergrund der Auswahl sowie zur Rolle von Funktionswörtern bietet ein [Unterkapitel](https://www.humanitiesdataanalysis.org/stylometry/notebook.html#function-words) in der Einführung Humanities Data Analysis von Karsdorp, Kestemont und Riddell.

Die Python-Bibliothek [scikit-learn](https://scikit-learn.org/stable/#) bietet viele Funktionalitäten und Algorithmen für die Datenanalyse sowie für Machine Learning. Die Objektklasse [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer) ermöglicht es, die Häufigkeiten von Wörtern in einem Textkorpus zu ermitteln. Zugleich ermöglicht sie die Tokenisierung von Texten, die hier im nachfolgenden Code über einen als Parameter übergebenen regulären Ausdruck erfolgt. Im Ergebnis erstellt der CountVectorizer auf der Basis des übergebenen Vokabulars eine Matrix mit Wordhäufigkeiten.

In [ ]:
vocab = [l.strip() for l in open('data/wordlist.txt') if not l.startswith('#') and l.strip()][:65]

In [63]:
vectorizer = text.CountVectorizer(token_pattern=r"(?u)\b\w+\b", vocabulary=vocab)
v_documents = vectorizer.fit_transform(documents).toarray()

print(v_documents.shape)
print(vectorizer.get_feature_names_out()[:5])

(36, 65)
['et' 'qui' 'in' 'non' 'ad']


In [39]:
print(v_documents[0:5])

[[347 369 170 289  87  94 100  66 105  59  91  86  45 108  24  32  53  24
   51  26  49  23  38  27  17  12   9  24  15  20   6  34   3   6  10  37
   15  10  30  23  13  20  18  12  21   3   2   2   8   7   2  10   9  13
   16   7  12  11   1  10   3  12  12   3  16]
 [409 367 222 272  68 107 124  97  87  55  75  94  68  84  23  51  34  41
   32  31  36  31  20  29  20  15  16  36  22  31  12  30   8  10  14  32
    7   9  32  15  18  25  26  26  28   7   5  12  10   9   2   8   7  12
    9   3   8   5   2   6   3   2  18   8   8]
 [392 335 206 203  84 126  87 110 109  53  65  86  76 113  29  31  34  38
   31  78  42  21  40  19  20  18  15  29  25  22   8  36  21  18  22  37
   12  10  39  27   7  14  10  19  10  11   1   6  10   9  14   8  13   7
   11   4   8   5   2   6   5  15  17  12   7]
 [367 334 192 250  61  96 126 106 102  55  67 115  73  96  27  43  49  51
   35  34  45  31  24  31  28   8  23  40  35  25   8  36  13  16  16  36
   23  16  36  26   9  23  20  23  17   4  10

### Normalisieren

Beim Normalisieren wird jedes Element des Dokumentvektors durch die Länge des Vektors, d.h. durch die Summe aller Elemente des Vektors, geteilt. Zur Anwendung kommt wiederum eine [Methode zur Normalisierung](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html?highlight=normalize#sklearn.preprocessing.normalize) aus [scikit-learn](https://scikit-learn.org/stable/#), die im konkreten Fall mit L1 normalisiert, was der beschriebenen Division durch die Vektorlänge entspricht.

In [54]:
print(f'Erstes Element aus dem ersten Dokumentenvektor: {v_documents[0][0]}')
print(f'Summe aller Elemente des ersten Dokumentenvektors: {v_documents[0].sum()}')
print(f'\nNormalisierter Wert des ersten Elements im ersten Dokumentenvektors: {v_documents[0][0] / v_documents[0].sum()}')

Erstes Element aus dem ersten Dokumentenvektor: 347
Summe aller Elemente des ersten Dokumentenvektors: 2877

Normalisierter Wert des ersten Elements im ersten Dokumentenvektors: 0.12061174834897463


In [65]:
v_documents = preprocessing.normalize(v_documents.astype(float), norm='l1')

print(v_documents.shape)

(36, 65)


In [61]:
v_documents[0:2]

array([[0.12061175, 0.1282586 , 0.05908933, 0.10045186, 0.03023983,
        0.03267292, 0.03475843, 0.02294056, 0.03649635, 0.02050747,
        0.03163017, 0.02989225, 0.01564129, 0.0375391 , 0.00834202,
        0.0111227 , 0.01842197, 0.00834202, 0.0177268 , 0.00903719,
        0.01703163, 0.00799444, 0.0132082 , 0.00938478, 0.00590893,
        0.00417101, 0.00312826, 0.00834202, 0.00521376, 0.00695169,
        0.00208551, 0.01181787, 0.00104275, 0.00208551, 0.00347584,
        0.01286062, 0.00521376, 0.00347584, 0.01042753, 0.00799444,
        0.0045186 , 0.00695169, 0.00625652, 0.00417101, 0.00729927,
        0.00104275, 0.00069517, 0.00069517, 0.00278067, 0.00243309,
        0.00069517, 0.00347584, 0.00312826, 0.0045186 , 0.00556135,
        0.00243309, 0.00417101, 0.00382343, 0.00034758, 0.00347584,
        0.00104275, 0.00417101, 0.00417101, 0.00104275, 0.00556135],
       [0.13484998, 0.12100231, 0.07319486, 0.08968018, 0.02242005,
        0.0352786 , 0.04088361, 0.03198154, 0.0

### Train-Test-Split

Um die Genauigkeit des Klassifizierungsalgorithmus zu bestimmen, werden die bekannten Textdaten in ein Testdatenset und ein Trainigsdatenset aufgeteilt. Die Größe der Testdaten ist hier die doppelte Anzahl der Autoren. 

In [11]:
test_size = len(set(authors)) * 2

(train_documents, test_documents,train_authors, test_authors) = model_selection.train_test_split(v_documents, 
                                                                                                 authors, 
                                                                                                 test_size=test_size, 
                                                                                                 stratify=authors, 
                                                                                                 random_state=42)
                                                                                                 
print(f'N={test_documents.shape[0]} Test Dokumente mit '
      f'V={test_documents.shape[1]} Features.')

print(f'N={train_documents.shape[0]} Training Dokumente mit '
      f'V={train_documents.shape[1]} Features.')

N=6 Test Dokumente mit V=65 Features.
N=30 Training Dokumente mit V=65 Features.


### Delta Klassifikator

In der nachfolgenden Python Objekt-Klasse wird ein Klassifikator erstellt, der die algorithmischen Berechnungen nach Vorgabe von Burrows Delta ausführt. Die Normalisierung der Vektoren ist bereits erfolgt. Zudem erfordert Burrows Delta eine Standardisierung. Die Klassenmethode *fit* übernimmt diesen Vorgang und nutzt dazu wiederum die Objektklasse [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html?highlight=standardscaler#sklearn.preprocessing.StandardScaler) aus scikit-learn. 

Die Klassenmethode *predict* berechnet die paarweisen Distanzen zwischen den Elementen zweier Sammlungen auf der Basis einer Distanzmetrik. Verwendung findet hier die [cdist()-Methode](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html) aus [SciPy](https://docs.scipy.org/doc/scipy/index.html#) einer weiteren Python-Bibliothek. Der voreingestellte Wert ist *cityblock*. Die Methode gibt als Vorhersage das Label (hier: den Autor) des Trainingsdokuments mit der geringsten Distanz zu dem Testdokument zurück, um auch das Testdokument mit diesem Label zu klassifizieren.

In [12]:
class Delta:
    """Delta-Based Authorship Attributer."""

    def fit(self, X, y):
        """Fit (or train) the attributer.

        Arguments:
            X: a two-dimensional array of size NxV, where N represents
               the number of training documents, and V represents the
               number of features used.
            y: a list (or NumPy array) consisting of the observed author
                for each document in X.

        Returns:
            Delta: A trained (fitted) instance of Delta.

        """
        self.train_y = np.array(y)
        self.scaler = preprocessing.StandardScaler(with_mean=False)
        self.train_X = self.scaler.fit_transform(X)

        return self

    def predict(self, X, metric='cityblock'):
        """Predict the authorship for each document in X.

        Arguments:
            X: a two-dimensional (sparse) matrix of size NxV, where N
               represents the number of test documents, and V represents
               the number of features used during the fitting stage of
               the attributer.
            metric (str, optional): the metric used for computing
               distances between documents. Defaults to 'cityblock'.

        Returns:
            ndarray: the predicted author for each document in X.

        """
        X = self.scaler.transform(X)
        dists = scidist.cdist(X, self.train_X, metric=metric)
        return self.train_y[np.argmin(dists, axis=1)]

### Evaluation 

Der Delta Klassifikator wird auf das Testsetting angewandt. Mit Hilfe einer bei scikit-learn verfügbaren Methode wird die [Accuracy-Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html?highlight=accuracy#sklearn.metrics.accuracy_score) des Klassifikators ermittelt.

In [14]:
delta = Delta()                             # Delta Classifier wird instantiiert
delta.fit(train_documents, train_authors)   # Delta Classifier wird trainiert
preds = delta.predict(test_documents)       # Delta Classifier klassifziert Test Dokumente

# Erstellen der Print-Ausgabe
for true, pred in zip(test_authors, preds):
    _connector = 'ABER' if true != pred else 'und'
    print(f'Der Autor ist {true} {_connector} {pred} wurde vorhergesagt.')

accuracy = metrics.accuracy_score(preds, test_authors)
print(f"\nAccuracy der Vorhersagen: {accuracy:.1f}")

Der Autor ist B und B wurde vorhergesagt.
Der Autor ist B und B wurde vorhergesagt.
Der Autor ist B und B wurde vorhergesagt.
Der Autor ist H und H wurde vorhergesagt.
Der Autor ist G und G wurde vorhergesagt.
Der Autor ist G und G wurde vorhergesagt.

Accuracy der Vorhersagen: 1.0


## Authorship Attribution: Anwendung auf den konkreten Untersuchungsfall

### Einlesen der Trainingsdaten

Die Länge des kürzesten zu prüfenden Quellendokuments ist 3.301 Tokens. Alle anderen Texte der Vergleichskorpora werden beim Einlesen auf diese Länge gebracht.

In [23]:
train_documents, train_authors, train_titles = load_directory('data/texts', 3301)

vectorizer = text.CountVectorizer(token_pattern=r"(?u)\b\w+\b", 
                                  vocabulary=vocab)
                                  
v_train_documents = vectorizer.fit_transform(train_documents).toarray()
v_train_documents = preprocessing.normalize(v_train_documents.astype(float), norm='l1')

delta = Delta().fit(v_train_documents, train_authors)

###  Einlesen der Testdaten

In [24]:
test_docs, test_authors, test_titles = load_directory('data/texts/test', 3301)

v_test_docs = vectorizer.transform(test_docs).toarray()
v_test_docs = preprocessing.normalize(v_test_docs.astype(float), norm='l1')

### Klassifizieren mit Cityblock-Metrik

In [27]:
predictions = delta.predict(v_test_docs)

for filename, prediction in zip(test_titles, predictions):
    print(f'Quelle: {filename} => klassifiziert als {prediction}')

Quelle: Mart-1 => klassifiziert als B
Quelle: Mart-1 => klassifiziert als G
Quelle: Missa-1 => klassifiziert als G
Quelle: Missa-2 => klassifiziert als G


### Klassifizieren mit Cosinus-Metrik

Die Klassifizierung unter Einbezug der Cosinus-Distanz ist bei stilometrischen Analysen zur Authorship Attribution ebenfalls solide und robust.

In [28]:
predictions = delta.predict(v_test_docs, metric='cosine')

for filename, prediction in zip(test_titles, predictions):
    print(f'Quelle: {filename} => klassifiziert als {prediction}')

Quelle: Mart-1 => klassifiziert als B
Quelle: Mart-1 => klassifiziert als G
Quelle: Missa-1 => klassifiziert als G
Quelle: Missa-2 => klassifiziert als G


## Ergebnis

Die Untersuchung mit Burrows Delta hat ergeben, dass keine der untersuchten Quellen eine Autorenschaft von Hildegard von Bingen nahelegt. Stattdessen kommt eher eine Autorenschaft von Guibert von Gembloux in Frage.